In [6]:
"""
import csv
import cv2
import numpy as np


lines = []
with open('./data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

lines.pop(0)

images = []
measurements = []
for line in lines:
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = './data/IMG/' + filename
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)
    
augmented_images = []
augmented_measurements = []
for image, measurement in zip(images,measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurements.append(measurement*-1.0)
    
X_data = np.array(augmented_images)
Y_data = np.array(augmented_measurements)
"""

"\nimport csv\nimport cv2\nimport numpy as np\n\n\nlines = []\nwith open('./data/driving_log.csv') as csvfile:\n    reader = csv.reader(csvfile)\n    for line in reader:\n        lines.append(line)\n\nlines.pop(0)\n\nimages = []\nmeasurements = []\nfor line in lines:\n    source_path = line[0]\n    filename = source_path.split('/')[-1]\n    current_path = './data/IMG/' + filename\n    image = cv2.imread(current_path)\n    images.append(image)\n    measurement = float(line[3])\n    measurements.append(measurement)\n    \naugmented_images = []\naugmented_measurements = []\nfor image, measurement in zip(images,measurements):\n    augmented_images.append(image)\n    augmented_measurements.append(measurement)\n    augmented_images.append(cv2.flip(image,1))\n    augmented_measurements.append(measurement*-1.0)\n    \nX_data = np.array(augmented_images)\nY_data = np.array(augmented_measurements)\n"

In [7]:
import sklearn
import csv
import cv2
import numpy as np


samples = []
with open('./data/driving_log_merged.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

samples.pop(0)

def generator(samples, batch_size=64):
    num_samples = len(samples)
    correction = 0.22
    while 1: # Loop forever so the generator never terminates
        sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            images = []
            angles = []
            for batch_sample in batch_samples:
                center_name = './data/IMG/'+batch_sample[0].split('/')[-1]
                left_name = './data/IMG/'+batch_sample[1].split('/')[-1]
                right_name = './data/IMG/'+batch_sample[2].split('/')[-1]
                center_image = cv2.imread(center_name)
                left_image = cv2.imread(left_name)
                right_image = cv2.imread(right_name)
                center_angle = float(batch_sample[3])
                left_angle = center_angle + correction
                right_angle = center_angle - correction
                images.append(center_image)
                angles.append(center_angle)
                images.append(cv2.flip(center_image,1))
                angles.append(center_angle*-1.0)
                images.append(left_image)
                angles.append(left_angle)
                images.append(cv2.flip(left_image,1))
                angles.append(left_angle*-1.0)                
                images.append(right_image)
                angles.append(right_angle)
                images.append(cv2.flip(right_image,1))
                angles.append(right_angle*-1.0)

            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

In [8]:
from sklearn.model_selection import train_test_split

train_samples, valid_samples = train_test_split(samples, test_size=0.2)
train_generator = generator(train_samples, batch_size=512)
valid_generator = generator(valid_samples, batch_size=512)

In [9]:
#import matplotlib.pyplot as plt
#plt.imshow(images[0])
#print(np.shape(X_train[0]))
#print(list(train_generator))

In [13]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, BatchNormalization, Activation
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()

model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))

model.add(Convolution2D(24,5,5,subsample=(2,2)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Convolution2D(36,5,5,subsample=(2,2)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Convolution2D(48,5,5,subsample=(2,2)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Convolution2D(64,3,3))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Convolution2D(64,3,3))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Flatten())

model.add(Dense(100))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(50))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(10))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
#model.fit(X_train, y_train, validation_split=0.2, shuffle=True)
model.fit_generator(train_generator, samples_per_epoch=len(train_samples)*6, validation_data=valid_generator, 
                    nb_val_samples=len(valid_samples)*6, nb_epoch=15)



model.save('model.h5')

Epoch 1/15
56322/56322 [==============================] - 50s - loss: 0.1753 - val_loss: 0.0537
Epoch 2/15
56322/56322 [==============================] - 48s - loss: 0.0330 - val_loss: 0.0502
Epoch 3/15
56322/56322 [==============================] - 47s - loss: 0.0273 - val_loss: 0.0501
Epoch 4/15
56322/56322 [==============================] - 47s - loss: 0.0249 - val_loss: 0.0509
Epoch 5/15
56322/56322 [==============================] - 47s - loss: 0.0235 - val_loss: 0.0520
Epoch 6/15
56322/56322 [==============================] - 48s - loss: 0.0222 - val_loss: 0.0535
Epoch 7/15
56322/56322 [==============================] - 48s - loss: 0.0213 - val_loss: 0.0556
Epoch 8/15
56322/56322 [==============================] - 54s - loss: 0.0204 - val_loss: 0.0580
Epoch 9/15
56322/56322 [==============================] - 55s - loss: 0.0196 - val_loss: 0.0586
Epoch 10/15
56322/56322 [==============================] - 53s - loss: 0.0190 - val_loss: 0.0567
Epoch 11/15
56322/56322 [==============